In [1]:
%matplotlib inline

In [2]:
import torch
import numpy as np

# RNN 실습

단어를 분류하기 위해 기초적인 문자-단위 RNN을 구축하고 학습 할 예정입니다.
이 튜토리얼에서는 (이후 2개 튜토리얼과 함께) NLP 모델링을 위한 데이터 전처리를
`torchtext` 의 편리한 많은 기능들을 사용하지 않고 어떻게 하는지 "기초부터(from scratch)"
보여주기 떄문에  NLP 모델링을 위한 전처리가 저수준에서 어떻게 진행되는지를 알 수 있습니다.
문자-단위 RNN은 단어를 문자의 연속으로 읽어 들여서 각 단계의 예측과
"은닉 상태(Hidden State)" 출력하고, 다음 단계에 이전 은닉 상태를 전달합니다.
단어가 속한 클래스로 출력이 되도록 최종 예측으로 선택합니다.

## 입력 데이터

하나의 문자를 표현하기 위해, 크기가 ``<1 x n_letters>`` 인
"One-Hot 벡터" 를 사용합니다. One-Hot 벡터는 현재 문자의
주소에만 1을 값으로 가지고 그외에 나머지는 0으로 채워집니다.  
예시 ``"b" = <0 1 0 0 0 ...>`` .

단어를 만들기 위해 One-Hot 벡터들을 2 차원 행렬
``<line_length x 1 x n_letters>`` 에 결합시킵니다.

위에서 보이는 추가적인 1차원은 PyTorch에서 모든 것이 배치(batch)에 있다고 가정하기
때문에 발생합니다.

In [3]:
# singleton example
# shape : (1, 1, 4)
# input_data_np = np.array([[[1, 0, 0, 0]]])

# sequential example
# shape : (3, 5, 4)
h = [1., 0., 0., 0.]
e = [0., 1., 0., 0.]
l = [0., 0., 1., 0.]
o = [0., 0., 0., 1.]

In [4]:
input_data = torch.tensor([[h, e, l, l, o], [e, o, l, l, l], [l, l, e, e, l]])

In [5]:
input_data

tensor([[[1., 0., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 0., 1.]],

        [[0., 1., 0., 0.],
         [0., 0., 0., 1.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 0., 1., 0.]],

        [[0., 0., 1., 0.],
         [0., 0., 1., 0.],
         [0., 1., 0., 0.],
         [0., 1., 0., 0.],
         [0., 0., 1., 0.]]])

## RNN 선언

Autograd 전에, Torch에서 RNN(recurrent neural network) 생성은
여러 시간 단계 걸처서 계층의 매개변수를 복제하는 작업을 포함합니다.
계층은 은닉 상태와 변화도(Gradient)를 가지며, 이제 이것들은 그래프 자체에서
완전히 처리됩니다.

(ref : https://pytorch.org/docs/stable/generated/torch.nn.RNN.html)

![rnn_caluclate](../_static/rnn_calculate.png)

(ref: https://ratsgo.github.io/natural%20language%20processing/2017/03/09/rnnlstm/)

In [7]:
# RNN 선언
rnn = torch.nn.RNN(input_size=4, hidden_size=2, num_layers=1)

In [8]:
rnn

RNN(4, 2)

RNN 객체가 보유하고 있는 모든 attribute 호출

In [9]:
rnn.__dict__

{'nonlinearity': 'tanh',
 'training': True,
 '_parameters': OrderedDict([('weight_ih_l0',
               Parameter containing:
               tensor([[-0.0053,  0.3793, -0.5820, -0.5204],
                       [-0.2723,  0.1896, -0.0140,  0.5607]], requires_grad=True)),
              ('weight_hh_l0',
               Parameter containing:
               tensor([[-0.0628,  0.1871],
                       [-0.2137, -0.1390]], requires_grad=True)),
              ('bias_ih_l0',
               Parameter containing:
               tensor([-0.6755, -0.4683], requires_grad=True)),
              ('bias_hh_l0',
               Parameter containing:
               tensor([-0.2915,  0.0262], requires_grad=True))]),
 '_buffers': OrderedDict(),
 '_non_persistent_buffers_set': set(),
 '_backward_hooks': OrderedDict(),
 '_is_full_backward_hook': None,
 '_forward_hooks': OrderedDict(),
 '_forward_pre_hooks': OrderedDict(),
 '_state_dict_hooks': OrderedDict(),
 '_load_state_dict_pre_hooks': OrderedDict(),

In [10]:
for param in iter(next(rnn.parameters())):
    print(param)  # RNN의 파라미터만 출력

tensor([-0.0053,  0.3793, -0.5820, -0.5204], grad_fn=<UnbindBackward>)
tensor([-0.2723,  0.1896, -0.0140,  0.5607], grad_fn=<UnbindBackward>)


## Multi-layers RNN

다층 RNN 선언 (num_layers 로 레이어의 개수 설정 가능)

In [11]:
rnn_multi_layers = torch.nn.RNN(input_size=4, hidden_size=2, num_layers=3)

In [12]:
rnn_multi_layers

RNN(4, 2, num_layers=3)

In [13]:
rnn_multi_layers._parameters

OrderedDict([('weight_ih_l0',
              Parameter containing:
              tensor([[ 0.2795,  0.4243, -0.4794, -0.3079],
                      [ 0.2568,  0.5872, -0.1455,  0.5291]], requires_grad=True)),
             ('weight_hh_l0',
              Parameter containing:
              tensor([[-0.1140,  0.0748],
                      [ 0.6403, -0.6560]], requires_grad=True)),
             ('bias_ih_l0',
              Parameter containing:
              tensor([-0.4452, -0.1790], requires_grad=True)),
             ('bias_hh_l0',
              Parameter containing:
              tensor([-0.2756,  0.6109], requires_grad=True)),
             ('weight_ih_l1',
              Parameter containing:
              tensor([[-0.4583, -0.3255],
                      [-0.4940, -0.6622]], requires_grad=True)),
             ('weight_hh_l1',
              Parameter containing:
              tensor([[-0.4128,  0.6078],
                      [ 0.3155,  0.3427]], requires_grad=True)),
             ('bia

## 입출력 확인

### Single Layer

입력 데이터에 대한 RNN의 output 및 히든 레이어의 아웃풋 (hidden state) 확인

In [14]:
outputs, hidden_state = rnn(input_data)

In [15]:
print(outputs)
print(outputs.size())

tensor([[[-0.7497, -0.6135],
         [-0.5282, -0.2473],
         [-0.9136, -0.4269],
         [-0.9136, -0.4269],
         [-0.9028,  0.1180]],

        [[-0.5753, -0.0070],
         [-0.9052,  0.2597],
         [-0.9173, -0.1989],
         [-0.9173, -0.1989],
         [-0.8996, -0.2725]],

        [[-0.9077, -0.3205],
         [-0.8944, -0.2902],
         [-0.5134, -0.0288],
         [-0.5134, -0.0288],
         [-0.9127, -0.2222]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


In [16]:
print(hidden_state)
print(hidden_state.size())

tensor([[[-0.9077, -0.3205],
         [-0.8944, -0.2902],
         [-0.5134, -0.0288],
         [-0.5134, -0.0288],
         [-0.9127, -0.2222]]], grad_fn=<StackBackward>)
torch.Size([1, 5, 2])


### Multi Layers

입력 데이터에 대한 다층 RNN의 output 및 히든 레이어의 아웃풋 (hidden state) 확인

In [17]:
outputs, hidden_state = rnn_multi_layers(input_data)

In [18]:
print(outputs)
print(outputs.size())

tensor([[[ 0.7543, -0.5375],
         [ 0.7845, -0.5849],
         [ 0.6533, -0.3864],
         [ 0.6533, -0.3864],
         [ 0.7243, -0.4933]],

        [[ 0.7869, -0.7195],
         [ 0.7127, -0.6512],
         [ 0.5887, -0.4131],
         [ 0.5887, -0.4131],
         [ 0.5597, -0.4298]],

        [[ 0.6109, -0.6075],
         [ 0.5860, -0.5538],
         [ 0.7732, -0.6762],
         [ 0.7732, -0.6762],
         [ 0.6341, -0.5182]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


In [19]:
print(hidden_state)
print(hidden_state.size())

tensor([[[-0.8182, -0.0702],
         [-0.7993, -0.3433],
         [-0.2320,  0.6506],
         [-0.2320,  0.6506],
         [-0.8197,  0.1777]],

        [[ 0.1845, -0.7633],
         [ 0.1997, -0.6192],
         [-0.2353, -0.8978],
         [-0.2353, -0.8978],
         [ 0.1801, -0.6859]],

        [[ 0.6109, -0.6075],
         [ 0.5860, -0.5538],
         [ 0.7732, -0.6762],
         [ 0.7732, -0.6762],
         [ 0.6341, -0.5182]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


------------------------------------------------------------------------------------------




# Practice

#### Q1. RNN 모듈을 LSTM 으로 변경하시오.

LSTM 모듈에 대한 ref

(ref : https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html)

In [25]:
# LSTM 선언
lstm = torch.nn.??(input_size=4, hidden_size=2, num_layers=3)

In [21]:
lstm

LSTM(4, 2, num_layers=3)

In [22]:
outputs, (hidden_state, _) = lstm(input_data)

In [23]:
# output 호출
print(outputs)
print(outputs.size())

tensor([[[-0.0684,  0.1712],
         [-0.0680,  0.1788],
         [-0.0682,  0.1728],
         [-0.0682,  0.1728],
         [-0.0681,  0.1772]],

        [[-0.0879,  0.2323],
         [-0.0883,  0.2345],
         [-0.0879,  0.2272],
         [-0.0879,  0.2272],
         [-0.0881,  0.2290]],

        [[-0.0939,  0.2460],
         [-0.0941,  0.2466],
         [-0.0936,  0.2497],
         [-0.0936,  0.2497],
         [-0.0937,  0.2444]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


In [24]:
print(hidden_state)
print(hidden_state.shape)

tensor([[[ 0.0567,  0.0531],
         [ 0.0675,  0.0558],
         [ 0.0672, -0.1853],
         [ 0.0672, -0.1853],
         [ 0.0715,  0.0583]],

        [[ 0.0833, -0.1614],
         [ 0.0842, -0.1605],
         [ 0.0980, -0.1692],
         [ 0.0980, -0.1692],
         [ 0.0774, -0.1625]],

        [[-0.0939,  0.2460],
         [-0.0941,  0.2466],
         [-0.0936,  0.2497],
         [-0.0936,  0.2497],
         [-0.0937,  0.2444]]], grad_fn=<StackBackward>)
torch.Size([3, 5, 2])


------------------------------------------------------------------------------------------




__(ref: https://github.com/deeplearningzerotoall/PyTorch)__